In [80]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import folium

from geopy.geocoders import Nominatim
from geopy.distance import distance

## Geospatial data for population growth in African countries
This is a short script to process the Population Count, v4.11 (2000, 2005, 2010, 2015, 2020), with the gridded population of the world in 5km grids. You can find this dataset [here](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-rev11/data-download). This dataset is part of the Earth Engine datasets [here](https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Count?hl=en#citations). It could also be accessed through an API.

### References

Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Count, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H4JW8BX5. Accessed 24 Mar 2023.

In [ ]:
# Helper function
def find_point(lat, long, miles, bearing):
    return distance(miles=miles).destination((lat, long), bearing=bearing)

In [ ]:
# Importing the whole dataset and the country references
data_pop = 'gpw-v4-population-count-rev11_totpop_2pt5_min_nc/gpw_v4_population_count_rev11_2pt5_min.nc'
countries_df = pd.read_csv('gpw-v4-population-count-rev11_totpop_2pt5_min_nc/gpw_v4_national_identifier_grid_rev11_lookup.txt', sep="\t")

# The ds has 3 dimensions and has long x lat x rasters elements
ds = xr.open_dataset(data_pop)

# These are the raster names for clarity
raster_name = {1: 'Adjusted-Population-Count_2000',
               2: 'Adjusted-Population-Count_2005',
               3: 'Adjusted-Population-Count_2010',
               4: 'Adjusted-Population-Count_2015',
               5: 'Adjusted-Population-Count_2020',
               6: 'Data-Quality_Data-Context_2010',
               7: 'Data-Quality_Mean-Administrative-Unit-Area_2010',
               8: 'Data-Quality_Water-Mask_2010',
               9: 'Land-Area_2010',
               10: 'Water-Area_2010',
               11: 'National-Identifier-Grid_2010',
               12: 'Data-Code_2010',
               13: 'Input-Data-Year_2010',
               14: 'Input-Data-Level_2010',
               15: 'Input-Data-Sex_2010',
               16: 'Input-Data-Age_2010',
               17: 'Growth-Rate-Start-Year_2010',
               18: 'Growth-Rate-End-Year_2010',
               19: 'Growth_Rate-Admin-Level_2010',
               20: 'Year-most-recent-Census_2010'}

In [ ]:
# The smallest I can get is country level information, pre-filter of the cities
# I will get the countries from the decided top 20 cities 
countries_to_filter = ['AGO', 'CMR', 'CIV', 'EGY', 'ETH', 
                       'MAR', 'NGA', 'ZAF', 'SDN', 'TZA', 
                       'COD', 'MDG', 'KEN']
filtered_countries_df = countries_df[countries_df['ISOCODE'].isin(countries_to_filter)]
filtered_countries_list = filtered_countries_df['Value'].tolist()

In [ ]:
# Viewing the countries
filtered_countries_df

In [ ]:
# Raster 11 represents the country codes, so I filtered the countries in raster 11 and then broadcasted
# to get every piece of information for the countries
# This takes a minute or two running
filtered_ds = ds.sel(raster=11).isin(filtered_countries_list)
filtered2, _ = xr.broadcast(filtered_ds, ds)
ds_africa = ds.where(filtered2, drop=True)

In [ ]:
# Here I am just cleaning turning the ds into a df and cleaning it, dropping NAs, adding labels to rasters
Top_20_Africa_df = ds_africa.to_dataframe().dropna().reset_index()\
                            .rename(columns = {'Population Count, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes': 'value'})


In [ ]:
# Just doing a bit more cleaning
Top_20_Africa_df['raster_name'] = Top_20_Africa_df['raster'].apply(lambda x: raster_name[x])
Top_20_Africa_df2 = Top_20_Africa_df.drop(columns = ['raster'])\
                                    .pivot(index= ['latitude', 'longitude'],
                                           columns = ['raster_name'],
                                           values = ['value'])

# this is to paste the country names, I just did it quickly with dict and parallel computing because I
# am scared of joinig the tables
country_values = filtered_countries_df['Value'].tolist()
country_names = filtered_countries_df['NAME0'].tolist()
country_dic = dict(zip(country_values,country_names))

Top_20_Africa_df2[('value','Country_Name')] = Top_20_Africa_df2[('value','National-Identifier-Grid_2010')].apply(lambda x: country_dic[x])
Top_20_Africa_df2.columns = Top_20_Africa_df2.columns.map(lambda x: x[1])

In [ ]:
# Printing the document
Top_20_Africa_df2.to_csv('Top-20_population-count_2000-2020.csv')
Top_20_Africa_df2.head()

In [ ]:
Top_20_Africa_df2.query('Country_Name == "United Republic of Tanzania"')

In [ ]:
# Checking the population as of 2020 just to be cautious
Top_20_Africa_df2[['Country_Name', 'Adjusted-Population-Count_2020']].groupby('Country_Name').sum()

## Filtering by City

So, we have the country-level data but we want the specific top 20 cities. To filter it we will geolocate the cities

In [89]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")
locations_dict = {}
#Top cities
cities = ['Dar es-Salam, Coastal Zone, Tanzania', 'Kinsasa, República Democrática del Congo', 
          'City of Johannesburg, South Africa', 'Lagos, Nigeria', 'Luanda, Angola', 'Jartum, Sudán',
          'Abiyán, Costa de Marfil', 'Alexandria, Egypt', 'Addis Ababa, Ethiopia', 'Ciudad del Cabo, Sudáfrica',
          'Yaoundé, Cameroon', 'Kano, Nigeria', 'Ekurhuleni, South Africa', 'Douala, Cameroon',
          'Casablanca, Morocco', 'Ibadan, Nigeria', 'Antananarivo, Madagascar', 'Nairobi, Kenya']

for i in cities:
    location = geolocator.geocode(i)
    locations_dict[i] = (location.latitude, location.longitude)

In [90]:
locations_df = pd.DataFrame(locations_dict, index = ['latitude','longitude']).T\
                 .reset_index().rename(columns = {'index': 'City'})
miles=20

# With the function distance from geopy, I can find a lat long given a distance
locations_df['North_bound'] = locations_df[['latitude', 'longitude']].apply(lambda x: find_point(x['latitude'], x['longitude'], miles, 0).latitude, axis=1)
locations_df['South_bound'] = locations_df[['latitude', 'longitude']].apply(lambda x: find_point(x['latitude'], x['longitude'], miles, 180).latitude, axis=1)
locations_df['East_bound'] = locations_df[['latitude', 'longitude']].apply(lambda x: find_point(x['latitude'], x['longitude'], miles, 90).longitude, axis=1)
locations_df['West_bound'] = locations_df[['latitude', 'longitude']].apply(lambda x: find_point(x['latitude'], x['longitude'], miles, 270).longitude, axis=1)

In [91]:
# just printing the locations because we will use them later
locations_df.to_csv('city_coordinates.csv', index=False)

In [ ]:
list_dfs = []
names_html = []

for index, row in locations_df.iterrows():
    north_bound = row['North_bound']
    south_bound = row['South_bound']
    east_bound = row['East_bound']
    west_bound = row['West_bound']
    
    query = 'latitude <= @north_bound & latitude >= @south_bound & longitude <= @east_bound & longitude >= @west_bound'
    filtered_df = Top_20_Africa_df2.query(query)
    filtered_df['City'] = row['City']
    list_dfs.append(filtered_df)
    
    m = folium.Map([row['latitude'], row['longitude']], zoom_start=10)
    
    for index_2, row_2 in filtered_df.reset_index().iterrows():
        folium.CircleMarker(location=(row_2['latitude'], row_2['longitude']), radius = row_2['Adjusted-Population-Count_2020']/50000).add_to(m)
    
    name = f'images/population/{row["City"]}.html'
    names_html.append(name)
    m.save(name)

In [93]:
Top_20_Africa_filtered = pd.concat(list_dfs,axis=0)
Top_20_Africa_filtered.to_csv('Top-20_population-count_2000-2020.csv')

In [ ]:
#half_point = Top_20_Africa_df2.shape[0]//2

In [ ]:
#Top_20_Africa_df2.iloc[:half_point].to_csv('Top-20_population-count_2000-2020_part1.csv')
#Top_20_Africa_df2.iloc[half_point:].to_csv('Top-20_population-count_2000-2020_part2.csv')

### This is just for checking specific countries

In [ ]:
from operator import itemgetter

In [ ]:
indexes_list = Top_20_Africa_df2.query('Country_Name == "United Republic of Tanzania"').index

In [ ]:
print(f'Max lat: {max(indexes_list, key = itemgetter(0))[0]}')
print(f'Min lat: {min(indexes_list, key = itemgetter(0))[0]}')
print(f'Max long: {max(indexes_list, key = itemgetter(1))[0]}')
print(f'Min long: {min(indexes_list, key = itemgetter(1))[0]}')

In [ ]:
Top_20_Africa_df2.query('Country_Name == "United Republic of Tanzania"')